In [ ]:
# !pip install autoscraper python-dotenv pydash yarl

In [1]:
from dotenv import load_dotenv
import os
import requests
from pydash import py_
from yarl import URL
import pandas as pd

In [2]:
load_dotenv()
DIFFBOT_TOKEN = os.getenv("DIFFBOT_TOKEN")

In [3]:
from autoscraper import AutoScraper

url = "https://www.discoverdatascience.org/programs/data-science-bootcamps/"

wanted_list = ["Big Data Bootcamp"]

scraper = AutoScraper()
bootcamps1 = scraper.build(url, wanted_list)

bootcamps1.remove("Iris dataset")

In [4]:
url = "https://www.switchup.org/rankings/best-data-science-bootcamps"

wanted_list = ["WeCloudData"]

bootcamps2 = scraper.build(url, wanted_list)

In [57]:
bootcamps = set(bootcamps1).union(set(bootcamps2))

In [58]:
bootcamps = list(map(lambda x: x.lower(), bootcamps))

In [43]:
def download_file(query: str, size: int = 25, extract_data: bool = True):
    """Download data from Diffbot"""
    query_string = f"type=query&token={DIFFBOT_TOKEN}&query={query}&size={size}"
    url = URL.build(
        scheme="https",
        host="kg.diffbot.com",
        path="/kg/dql_endpoint",
        query_string=query_string,
    )
    r = requests.get(url)
    response = r.json()
    return response["data"] if extract_data else response

In [49]:
# ds_bootcamps = download_file(
#     f'type:Person educations.institution.name:or{str(tuple(bootcamps))} employments.title:or("data scientist") facet:educations.institution.name',
#     size=3423,
# )

# ml_bootcamps = download_file(
#     f'type:Person educations.institution.name:or{str(tuple(bootcamps))} employments.title:or("machine learning engineer") facet:educations.institution.name',
#     size=661,
# )

# de_bootcamps = download_file(
#     f'type:Person educations.institution.name:or{str(tuple(bootcamps))} employments.title:or("data engineer") facet:educations.institution.name',
#     size=976,
# )

In [103]:
import json

names = ['ds_bootcamps', 'ml_bootcamps', 'de_bootcamps']
# data = [ds_bootcamps, ml_bootcamps, de_bootcamps]

# for name, d in zip(names, data):
#     with open(name + '.json', "w") as f:
#         json.dump(d, f)

data = {}
for name in names:
    with open(name + '.json', "r") as f:
        data[name] = json.load(f)


In [104]:
import plotly.express as px


def get_dataframe(data: list, name: str, bootcamps: list):
    data = data[name]
    education = py_.map(data, "value")
    count = py_.map(data, "count")
    df = pd.DataFrame({"education": education, 'value': count})
    df = df[df.education.isin(bootcamps)]
    df['position'] = name
    return df


def plot_top_bootcamps(data: list, bootcamps: list):
    df = get_dataframe(data, bootcamps)
    fig = px.bar(df, x="education", y="count")
    fig.show()


In [105]:
df = pd.concat([get_dataframe(data, "de_bootcamps", bootcamps),
           get_dataframe(data, "ds_bootcamps", bootcamps),
           get_dataframe(data, "ml_bootcamps", bootcamps),
           ])


In [82]:
pivot = pd.pivot_table(df, values='value', index='education', columns='position').fillna(0)

In [98]:
names_map = {'de_bootcamps': 'data engineer',
             'ds_bootcamps': 'data scientist',
             'ml_bootcamps': 'machine learning engineer'}
pivot = pivot.reset_index().rename(columns=names_map)


In [102]:
px.bar(pivot.reset_index(), x='education', y=list(names_map.values())).update_xaxes(
    categoryorder='total descending')
